In [1]:
#pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
#pip install google-api-python-client

In [3]:
#pip install plyer

In [4]:
#pip install isodate
#Necessário para alterar o farmato da duração do vídeo 

In [5]:

from googleapiclient.discovery import build
import pandas as pd
import requests 
import isodate
import sqlite3

In [6]:
url = "https://www.youtube.com/@depoisdosquinze"
response = requests.get(url)
response

<Response [200]>

In [7]:
url = "https://www.youtube.com/@fabisantina"
response = requests.get(url)
response

<Response [200]>

In [8]:
url = "https://www.youtube.com/@TacieleAlcoleaa"
response = requests.get(url)
response

<Response [200]>

In [9]:
#Substitua 'YOUR_API_KEY' pela sua chave de API do YouTube
api_key = 'YOUR_API_KEY'

In [10]:
# Construa o serviço da API do YouTube
youtube = build('youtube', 'v3', developerKey=api_key)

def get_channel_id(youtube, custom_url):
    request = youtube.search().list(
                part='snippet',
                q=custom_url,
                type='channel',
                maxResults=1)
    response = request.execute()
    
    for item in response['items']:
        return item['snippet']['channelId']
    
    return None



Para utlizar a API do Youtube é nessário extrair o ID dos canais escolhidos

In [11]:
# URL personalizada do canal " Depois dos quinze"
custom_url = 'https://www.youtube.com/@depoisdosquinze'

# Obter ID do canal 
channel_id = get_channel_id(youtube, custom_url)
print(f"ID do canal Depois dos Quinze: {channel_id}")

ID do canal Depois dos Quinze: UCNWlx_0XxpUkci6PnCwH_dQ


In [12]:
# URL personalizada do canal "Taciele Alcolea"
custom_url = 'https://www.youtube.com/@TacieleAlcoleaa'

# Obter ID do canal 
channel_id = get_channel_id(youtube, custom_url)
print(f"ID do canal Taciele Alcolea: {channel_id}")

ID do canal Taciele Alcolea: UCy9xDEfMD8jrXU3NatFLLtw


In [13]:
# URL personalizada do canal "Fabi Santina"
custom_url = 'https://www.youtube.com/@fabisantina'

# Obter ID do canal 
channel_id = get_channel_id(youtube, custom_url)
print(f"ID do canal Fabi Santina: {channel_id}")

ID do canal Fabi Santina: UC5qJINQ80-zvtGQ4fq3kSRQ


In [14]:
# Construa o serviço da API do YouTube
youtube = build('youtube', 'v3', developerKey=api_key)


In [15]:
# IDs de canais para análise (substitua pelos IDs dos canais desejados)
channel_ids = ['UCy9xDEfMD8jrXU3NatFLLtw', 'UCNWlx_0XxpUkci6PnCwH_dQ', 'UC5qJINQ80-zvtGQ4fq3kSRQ']

In [16]:
# Inicializar uma lista para coletar detalhes dos vídeos de todos os canais
all_video_details = []
all_channel_activities = []

In [17]:
# Obter estatísticas dos canais
def get_channel_stats(youtube, channel_ids):
    all_data = []
    request = youtube.channels().list(
                part='snippet,contentDetails,statistics',
                id=','.join(channel_ids)) 
    response = request.execute()
    
    for item in response['items']:
        published_at = item['snippet'].get('publishedAt', 'N/A')
        
        if published_at != 'N/A':
            # Separando data e tempo
            date_part, time_part = published_at.split('T')
            time_part = time_part.replace('Z', '')  # Remover o 'Z' do tempo
            date_part = pd.to_datetime(date_part).strftime('%d/%m/%Y')  # Formatar data para dd/mm/yyyy

        else:
            date_part = 'N/A'
            time_part = 'N/A'
        
        data = {
            'channel_id': item['id'],
            'channel_name': item['snippet']['title'],
            'published_date': date_part,
            'published_time': time_part,
            'country': item['snippet'].get('country', 'N/A'),
            'subscribers': int(item['statistics'].get('subscriberCount', 0)),
            'views': int(item['statistics'].get('viewCount', 0)),
            'total_videos': int(item['statistics'].get('videoCount', 0)),
            'playlist_id': item['contentDetails']['relatedPlaylists']['uploads']
        }
        all_data.append(data)
    
    return pd.DataFrame(all_data)


In [18]:
def get_video_ids(youtube, playlist_id):
    # Cria uma lista vazia para armazenar os IDs dos vídeos
    video_ids = []

    # Faz a primeira solicitação à API do YouTube para obter detalhes dos vídeos na playlist
    request = youtube.playlistItems().list(
                part='contentDetails',  # Pede para retornar apenas os detalhes do conteúdo
                playlistId=playlist_id,  
                maxResults=50)  # Número máximo de vídeos retornados em uma solicitação (máximo permitido é 50)
    response = request.execute()  # Executa a solicitação e armazena a resposta na variável 'response'
    
    # Loop através dos itens retornados na resposta
    for item in response['items']:
        # Adiciona o ID do vídeo à lista 'video_ids'
        video_ids.append(item['contentDetails']['videoId'])
    
    # Verifica se há mais páginas de resultados
    next_page_token = response.get('nextPageToken')
    while next_page_token:  
        request = youtube.playlistItems().list(
                    part='contentDetails',  
                    playlistId=playlist_id,  
                    maxResults=50,  
                    pageToken=next_page_token)  
        response = request.execute()  
        
        # Adiciona os IDs dos vídeos da nova página à lista 'video_ids'
        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])
        
        # Atualiza o token da próxima página (ou se não houver, termina o loop)
        next_page_token = response.get('nextPageToken')
    
    # Retorna a lista completa de IDs de vídeos
    return video_ids


In [19]:
def get_video_details(youtube, video_ids, channel_name):
    all_video_stats = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
                    part='snippet,statistics,contentDetails',
                    id=','.join(video_ids[i:i+50]))
        response = request.execute()
        
        for video in response['items']:
            # Converte a data e hora para o formato datetime
            published_at = pd.to_datetime(video['snippet']['publishedAt'])
            
            # Extrai a data e o horário em colunas separadas
            date = published_at.strftime('%d-%m-%Y')
            time = published_at.strftime('%H:%M:%S')
            
            # Converte a duração para o formato de horas:minutos:segundos
            duration_iso = video['contentDetails']['duration']
            duration = isodate.parse_duration(duration_iso)
            duration_str = str(duration)
            
            video_stats = {
                'channel_name': channel_name,
                'video_id': video['id'],
                'title': video['snippet']['title'],
                'published_date': date,  # Coluna com a data no formato dia-mês-ano
                'published_time': time,  # Coluna com o horário
                'duration': duration_str,  # Duração convertida para formato legível
                'category_id': video['snippet']['categoryId'],
                'views': int(video['statistics'].get('viewCount', 0)),
                'likes': int(video['statistics'].get('likeCount', 0)),
                'comments': int(video['statistics'].get('commentCount', 0))
            }
            all_video_stats.append(video_stats)
    
    # Retorna os dados como um DataFrame
    return pd.DataFrame(all_video_stats)


In [20]:
def get_channel_activity(youtube, playlist_id):
    # Obtém os detalhes dos vídeos
    video_details = get_video_details(youtube, get_video_ids(youtube, playlist_id), "")
    
    # Converte a coluna 'published_date' para o formato datetime
    video_details['published_date'] = pd.to_datetime(video_details['published_date'], format='%d-%m-%Y')
    
    
    # Calcula a frequência mensal de uploads
    # Extrai o mês e o ano
    video_details['month'] = video_details['published_date'].dt.strftime('%m-%Y')
    monthly_uploads = video_details.groupby(['channel_name', 'month']).size().reset_index(name='uploads')
        
    # Calcula o engajamento médio
    avg_engagement = video_details[['views', 'likes', 'comments']].mean().reset_index()

    # Arredonda as médias para 2 casas decimais
    avg_engagement.columns = ['metric', 'average']
    avg_engagement['average'] = avg_engagement['average'].round(2)
        
    return monthly_uploads, avg_engagement


In [21]:
# Obter estatísticas dos canais
channel_stats = get_channel_stats(youtube, channel_ids)

# Obter IDs dos vídeos e detalhes dos vídeos de cada canal
for index, row in channel_stats.iterrows():
    playlist_id = row['playlist_id']
    channel_name = row['channel_name']
    video_ids = get_video_ids(youtube, playlist_id)
    video_details = get_video_details(youtube, video_ids, channel_name)
    all_video_details.append(video_details)
    
    # Obter atividade do canal
    monthly_uploads, avg_engagement = get_channel_activity(youtube, playlist_id)
    
    # Arredondar as médias de engajamento para 2 casas decimais
    avg_engagement = avg_engagement.round(2)
    
    monthly_uploads['channel_name'] = channel_name
    avg_engagement['channel_name'] = channel_name
    all_channel_activities.append((monthly_uploads, avg_engagement))


In [22]:
# Combinar todos os detalhes dos vídeos em um único DataFrame
all_video_details_df = pd.concat(all_video_details, ignore_index=True)

# Combinar atividades dos canais
monthly_uploads_df = pd.concat([x[0] for x in all_channel_activities], ignore_index=True)#Mostra quantos vídeos foram postados em cada mês
avg_engagement_df = pd.concat([x[1] for x in all_channel_activities], ignore_index=True)#Mostra a média de visualizações, curtidas e comentários dos vídeos do canal


In [23]:
# Exibir o engajamento médio
avg_engagement_df.head(3)

,metric,average,channel_name
0,views,115288.37,Bruna Vieira
1,likes,8011.25,Bruna Vieira
2,comments,396.46,Bruna Vieira


In [53]:
avg_engagement_df.dtypes

metric           object
average         float64
channel_name     object
dtype: object

In [24]:
# Exibir os uploads mensais
monthly_uploads_df.head(3)

,channel_name,month,uploads
0,Bruna Vieira,01-2013,5
1,Bruna Vieira,01-2014,7
2,Bruna Vieira,01-2015,11


In [52]:
monthly_uploads_df.dtypes

channel_name    object
month           object
uploads          int64
dtype: object

In [25]:
# Exibir os detalhes dos vídeos
all_video_details_df.head(3)

,channel_name,video_id,title,published_date,published_time,duration,category_id,views,likes,comments
0,Bruna Vieira,-KjbksWBJUA,Diário da obra #02: tour pelo apê ANTES da que...,15-06-2024,18:00:00,0:11:39,19,25221,2185,143
1,Bruna Vieira,wjtD-gCBrUY,acompanhem o Diário Da Obra do Estúdio Bruvie ...,11-06-2024,10:25:49,0:00:27,19,6972,490,3
2,Bruna Vieira,xfwjE7pCgZY,Diário da obra #01: Estúdio Bruvie em São Paulo,09-06-2024,23:18:33,0:08:28,19,23977,2400,142


In [51]:
all_video_details_df.dtypes

channel_name      object
video_id          object
title             object
published_date    object
published_time    object
duration          object
category_id       object
views              int64
likes              int64
comments           int64
dtype: object

In [26]:
# Exibir os dados dos canais
channel_stats.head(5)

,channel_id,channel_name,published_date,published_time,country,subscribers,views,total_videos,playlist_id
0,UCNWlx_0XxpUkci6PnCwH_dQ,Bruna Vieira,15/11/2011,01:47:55,BR,1330000,95343145,826,UUNWlx_0XxpUkci6PnCwH_dQ
1,UCy9xDEfMD8jrXU3NatFLLtw,Taciele Alcolea,08/09/2009,15:51:09,BR,5860000,888883572,2275,UUy9xDEfMD8jrXU3NatFLLtw
2,UC5qJINQ80-zvtGQ4fq3kSRQ,Fabi Santina,02/10/2010,01:53:56,BR,1820000,330503095,2371,UU5qJINQ80-zvtGQ4fq3kSRQ


In [50]:
channel_stats.dtypes

channel_id        object
channel_name      object
published_date    object
published_time    object
country           object
subscribers        int64
views              int64
total_videos       int64
playlist_id       object
dtype: object

In [27]:
#Cria um arquivo xlsx com 4 abas e os dados extraídos
with pd.ExcelWriter('Análise_Youtube.xlsx') as writer:
    channel_stats.to_excel(writer, sheet_name='Estatísticas dos Canais', index=False)
    all_video_details_df.to_excel(writer, sheet_name='Detalhes dos Vídeos', index=False)
    monthly_uploads_df.to_excel(writer, sheet_name='Uploads Mensais', index=False)
    avg_engagement_df.to_excel(writer, sheet_name='Média de Engajamento', index=False)


In [28]:
from plyer import notification
notification.notify(
    title = "Projeto Final YouTube",
    message = "Dados exportados para 'Análise_Youtube.xlsx",
    app_name = 'Notificação do projeto',
    timeout = 10
)

In [37]:
conn = sqlite3.connect('Análise_Youtube.db')


In [38]:
channel_stats.to_sql('Estatísticas dos Canais', conn, if_exists='replace', index=False)
all_video_details_df.to_sql('Detalhes dos Vídeos', conn, if_exists='replace', index=False)
monthly_uploads_df.to_sql('Uploads Mensais', conn, if_exists='replace', index=False)
avg_engagement_df.to_sql('Média de Engajamento', conn, if_exists='replace', index=False)

9

In [42]:
query = "SELECT * FROM 'Estatísticas dos Canais'"
df = pd.read_sql(query, conn)
print(df)

                 channel_id     channel_name published_date published_time  \
0  UCNWlx_0XxpUkci6PnCwH_dQ     Bruna Vieira     15/11/2011       01:47:55   
1  UCy9xDEfMD8jrXU3NatFLLtw  Taciele Alcolea     08/09/2009       15:51:09   
2  UC5qJINQ80-zvtGQ4fq3kSRQ     Fabi Santina     02/10/2010       01:53:56   

  country  subscribers      views  total_videos               playlist_id  
0      BR      1330000   95343145           826  UUNWlx_0XxpUkci6PnCwH_dQ  
1      BR      5860000  888883572          2275  UUy9xDEfMD8jrXU3NatFLLtw  
2      BR      1820000  330503095          2371  UU5qJINQ80-zvtGQ4fq3kSRQ  


In [43]:
conn.commit()
conn.close()